# Fun Hunt in downtown Toronto (Part 2)

# Load Packages

In [1]:
!pip install folium

In [2]:
!pip install pgeocode

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim

In [4]:
def locator(df, postal_code, country_code):
    import pgeocode
    import pandas

    nomi = pgeocode.Nominatim(country_code)
    df['lat'] = df[postal_code].apply(lambda x:nomi.query_postal_code(x).latitude)
    df['lng'] = df[postal_code].apply(lambda x:nomi.query_postal_code(x).longitude)

    return df

In [5]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')
wikitable = soup.find('table')
columns = [i.text.strip() for i in wikitable.find_all('th')]
data = [[j.text.strip() for j in i.find_all('td')] for i in wikitable.find_all("tr")[1:]]
trtdata = pd.DataFrame(data, columns = columns)
trtdata['Neighbourhood'] = trtdata.Neighbourhood.apply(lambda x: x.replace(' /',','))
trtdata.columns = ['PostalCode','Borough', 'Neighborhood']

In [6]:
trtdata1 = trtdata.loc[trtdata.Borough != 'Not assigned'].reset_index(drop=True)
trtdata1.loc[trtdata1.Neighborhood == 'Not assigned', 'Neighborhood'] = trtdata['Borough']
trtdata2 = trtdata1.groupby(['PostalCode', 'Borough']).agg(', '.join).reset_index()
trtdata3 = locator(trtdata2, 'PostalCode', 'ca')
trtdata3 = trtdata3.dropna()

In [7]:
trtdata3.head()

,PostalCode,Borough,Neighborhood,lat,lng
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
radius=1000
LIMIT=10
VERSION = '20200605'
lat = trtdata3.lat[0]
lng = trtdata3.lng[0]

In [10]:
trtdata3.head()

,PostalCode,Borough,Neighborhood,lat,lng
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [77]:
trtdata3.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Etobicoke'], dtype=object)

In [83]:
bt_trt = trtdata3.loc[trtdata3['Borough'].isin(["Downtown Toronto", "Central Toronto", "North York"]),:].reset_index(drop=True)

In [84]:
bt_trt

,PostalCode,Borough,Neighborhood,lat,lng
0,M2H,North York,Hillcrest Village,43.8015,-79.3577
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.7801,-79.3479
2,M2K,North York,Bayview Village,43.7797,-79.3813
3,M2L,North York,"York Mills, Silver Hills",43.7547,-79.3764
4,M2M,North York,"Willowdale, Newtonbrook",43.7915,-79.4103
5,M2N,North York,"Willowdale, Willowdale East",43.7673,-79.4111
6,M2P,North York,York Mills West,43.7500,-79.3978
7,M2R,North York,"Willowdale, Willowdale West",43.7786,-79.4450
8,M3A,North York,Parkwoods,43.7545,-79.3300
9,M3B,North York,Don Mills,43.7450,-79.3590


In [85]:
def search(CLIENT_ID, CLIENT_SECRET, DATAFRAME, LIMIT=1000, VERSION = '20200605', RADIUS=1000):
    results = pd.DataFrame(columns=['neighborhood','borough', 'venue_name', 'venue_cat', 'venue_lat', 'venue_lng'])
    for row in range(DATAFRAME.shape[0]):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            DATAFRAME.lat[row], 
            DATAFRAME.lng[row], 
            RADIUS, 
            LIMIT)
    
        # make the GET request
#         print(url)
        result = requests.get(url).json()
        for ven in range(len(result['response']['groups'][0]['items'])):
            results = results.append(
                {'neighborhood': DATAFRAME.Neighborhood[row], 
                'borough': DATAFRAME.Borough[row],
                'venue_name': result['response']['groups'][0]['items'][ven]['venue']['name'],
                'venue_cat': result['response']['groups'][0]['items'][ven]['venue']['categories'][0]['name'],
                'venue_lat': result['response']['groups'][0]['items'][ven]['venue']['location']['lat'],
                'venue_lng': result['response']['groups'][0]['items'][ven]['venue']['location']['lng'],}, ignore_index=True
                )
    return results

In [87]:
results = search(CLIENT_ID, CLIENT_SECRET, DATAFRAME=bt_trt)

In [88]:
results

,neighborhood,borough,venue_name,venue_cat,venue_lat,venue_lng
0,Hillcrest Village,North York,Tastee,Bakery,43.807722,-79.356798
1,Hillcrest Village,North York,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,Korean Restaurant,43.798391,-79.369187
2,Hillcrest Village,North York,Tim Hortons,Coffee Shop,43.792905,-79.354490
3,Hillcrest Village,North York,Rexall Pharma Plus,Pharmacy,43.793962,-79.353240
4,Hillcrest Village,North York,TD Canada Trust,Bank,43.794247,-79.353239
5,Hillcrest Village,North York,RBC Royal Bank,Bank,43.793634,-79.353760
6,Hillcrest Village,North York,Tim Hortons,Coffee Shop,43.798945,-79.369644
7,Hillcrest Village,North York,Subway,Sandwich Place,43.794492,-79.353595
8,Hillcrest Village,North York,Listo's No Frills,Grocery Store,43.794373,-79.353754
9,Hillcrest Village,North York,TD Canada Trust,Bank,43.798466,-79.368832
